In [ ]:
import random
import numpy as np

In [ ]:
from Farm_Evaluator_Vec import getTurbLoc, loadPowerCurve, binWindResourceData, preProcessing, getAEP, checkConstraints

In [ ]:
#turb_coords   =  getTurbLoc(r'turbine_loc_test.csv')
power_curve   =  loadPowerCurve('power_curve.csv')
wind_inst_freq =  binWindResourceData(r'wind_data_2007.csv')
turb_rad = 50

In [ ]:
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)

In [ ]:
# Problem parameter
min_lim = 50
max_lim = 3950
turb_count = 50
penalty = 0.05

In [ ]:
def gen_coord():
    x = round(random.uniform(min_lim, max_lim),2)
    y = round(random.uniform(min_lim, max_lim),2)
    return x,y

In [ ]:
def check_violation(turb1, turb2):
    return np.linalg.norm(turb1 - turb2) > 8*turb_rad

In [ ]:
def generate_turb_locations():
    arr = [gen_coord()]
    for i in range(turb_count-1):
        while True:
            new_cord = gen_coord()

            if all([check_violation(np.array(cord), np.array(new_cord)) for cord in arr]):
                arr.append(new_cord)
                break
    return arr

In [ ]:
def calculate_AEP(turb_coords):
    return getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, 
                  n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)

In [ ]:
class TurbGroup:
    def __init__(self):
        self.turb_coords = np.array(generate_turb_locations())
        self.fitness = -1
        self.violation_count = 0
    
    def count_violation(self):
        for i,turb1 in enumerate(self.turb_coords):
            for turb2 in np.delete(self.turb_coords, i, axis=0):
                if  np.linalg.norm(turb1 - turb2) < 8*turb_rad:
                    self.violation_count += 1
                    
        
    def calculate_fitness(self):
        self.count_violation()
        fitness = calculate_AEP(self.turb_coords)
        fitness -= fitness*penalty*self.violation_count
        self.fitness = fitness


In [ ]:
# GA parameters
POP_SIZE = 100
GEN_COUNT = 50
TOURNAMENT_SIZE = 10
MUT_PROB = 0.05

In [ ]:
def ga():

    pop = init_pop(POP_SIZE)
    #best_in_each_gen = []
    
    for generation in range(GEN_COUNT):

        pop = fitness(pop)
        
        nextgen_pop = []
        for i in range(int(POP_SIZE/2)):
            parent1 = selection(pop)
            parent2 = selection(pop)
            offspring = crossover(parent1, parent2)
            nextgen_pop.append(mutation(offspring[0]))
            nextgen_pop.append(mutation(offspring[1]))
        pop = nextgen_pop
        best = sorted(pop, key=lambda x: x.fitness, reverse=True)
        print("Best fit in Generation no {}: {}".format(generation, best[0].fitness))


In [ ]:
def init_pop(population_count):
    return [TurbGroup() for _ in range(population_count)]

In [ ]:
def fitness(population):
    for agent in population:
        agent.calculate_fitness()
    return population

In [ ]:
def selection(population):
    
    players = random.sample(population, TOURNAMENT_SIZE)
    sorted_players = sorted(players, key=lambda x: x.fitness, reverse=True)

    return sorted_players[0]

In [ ]:
def crossover(p1, p2):
    xo_pt = random.randrange(turb_count)
    p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt] = p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt]
    return p1, p2

In [ ]:
def mutation(agent):
    if random.random() < MUT_PROB:
        return TurbGroup()
    else:
        return agent

In [ ]:
ga()

Best fit in Generation no 0: 515.38005569458
Best fit in Generation no 1: 515.38005569458
Best fit in Generation no 2: 515.38005569458
Best fit in Generation no 3: 515.38005569458
Best fit in Generation no 4: 515.38005569458
Best fit in Generation no 5: 516.0323840332031
Best fit in Generation no 6: 516.0323840332031
